In [1]:
print("Hello Landed")

Hello Landed


#### Imports
* sagemaker_pyspark -> sagemaker/spark interface
* pyspark -> Imports for spark ctrl

In [2]:
import sagemaker_pyspark
from pyspark import SparkContext

### Folders for storing files
* Using relative path
* land_script is already in landed folder

In [3]:
LANDED = '.'

#### Build and store Spark session

In [10]:
from sagemaker_pyspark import classpath_jars

classpath = ":".join(sagemaker_pyspark.classpath_jars())

from pyspark.sql import SparkSession
builder = SparkSession.builder.appName("MUSIC SPARK")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
# Unecessary setting to the default value
#builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
# TODO: What means debut.maxToStringFields?
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.extraClassPath", classpath)
# Unecessary setting the default value 
#builder.config("spark.driver.memory", "1g")
#builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")

spark = builder.getOrCreate()
spark

In [11]:
all_artist = [
    'https://soundcloud.com/soundcloud-hustle/sets/payday',
    'https://soundcloud.com/helennpham/sets/lofi-hip-hop',
    'https://soundcloud.com/lukasm1/sets/chill-mix-high-on-chill'
]

In [12]:
from soundscrape.soundscrape import process_soundcloud
def download_audio(url='', number=8, path=LANDED):
    vargs = {
        'path':path,
        'folders': False,
        'group': False,
        'track': '',
        'num_tracks': number,
        'bandcamp': False,
        'downloadable': False,
        'likes': True,
        'open': False,
        'artist_url': url,
        'keep': True}
    process_soundcloud(vargs)

In [13]:
pipe_music = spark.sparkContext.parallelize(all_artist).foreach(download_audio)